In [16]:
import pandas as pd
import numpy as np
import pickle as pkl
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input,GRU, SimpleRNN
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.metrics import AUC


In [2]:
data = pd.read_csv('games.csv')
moves = np.array(data.query("winner != 'draw'")['moves'])
labels = np.array(data.query("winner != 'draw'")['winner'].apply(lambda x: 1 if x =='white' else 0))
labels
all_moves = set()
for moves_list in moves:
    for move in moves_list.split(' '):
        if move not in all_moves:
            all_moves.add(move)


vocab_length = len(all_moves)
vocab_length

4373

In [3]:
moves.shape

(19108,)

In [5]:
max_moves = data.turns.max()
max_moves

349

In [6]:
print(vocab_length)
print(max_moves)

4373
349


In [7]:
tokenizer = Tokenizer(num_words=vocab_length)
tokenizer.fit_on_texts(moves)

sequences = tokenizer.texts_to_sequences(moves)
word_index = tokenizer.word_index

model_inputs = pad_sequences(sequences, maxlen=max_moves)

In [8]:
model_inputs.shape, labels.shape

((19108, 349), (19108,))

In [9]:
X_train,X_test,y_train,y_test = train_test_split(model_inputs, labels, test_size=0.3, random_state=24)

In [15]:
inputs = Input(shape=int(max_moves))
embedding = Embedding(input_dim=vocab_length, output_dim=256, input_length=max_moves)(inputs)
gru = GRU(units=256)(embedding)
outputs = Dense(1, activation="sigmoid")(gru)

model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])
history = model.fit(X_train, y_train, validation_split=0.2, epochs=3, batch_size=32, verbose=2, callbacks=[ReduceLROnPlateau()])